# Campervan Projekt

## Libraries and settings

In [49]:
# Libraries
import os
import re
import pprint
import requests
import numpy as np
import pandas as pd

import geopandas as gpd

from shapely.geometry import Polygon

import matplotlib.patches as patches
from matplotlib import pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/workspaces/project_Autoscout_Coding_Pros/Project_Campers


## Import data

In [50]:
# Read data
df_orig = pd.read_csv("./data/Autoscout24_Camper.csv", sep=",", encoding='utf-8')

# Show column names from df_orig
print(df_orig.columns)

# define the columns to keep
colums = ['web-scraper-order',
        'description_raw',
        'price_raw', 
        'mileage_raw',
       'fuel_type_raw', 
       'location_raw']

# keep only the columns we are interested in
df = df_orig[colums]

# Show first records of data frame
df.head()


Index(['web-scraper-order', 'web-scraper-start-url', 'autoscout24',
       'autoscout24-href', 'description_raw', 'price_raw', 'mileage_raw',
       'fuel_type_raw', 'location_raw'],
      dtype='object')


,web-scraper-order,description_raw,price_raw,mileage_raw,fuel_type_raw,location_raw
0,1733470796-1,MERCEDES-BENZ Camper,CHF 15'900.–,265'000 km,Diesel,"Mittelstrasse 38, 3613 Steffisburg"
1,1733470799-2,CARADO I 449 Édition 15 Fiat Capron 169L/1449,CHF 71'900.–,14'000 km,Diesel,"Zone industrielle Les Portettes 2c, 1312 Eclépens"
2,1733470801-3,DETHLEFFS Fiat Ducato,CHF 37'800.–,90'000 km,Diesel,"Rte du St-Bernard 8, 1937 Orsières"
3,1733470805-4,FIAT Euro Master Wohnmobil,CHF 19'900.–,187'236 km,Diesel,"Badenerstrasse 84, 8952 Schlieren"
4,1733470807-5,VW T6.1 California 2.0 TDI Ocean Edition Liber...,CHF 84'500.–,16'500 km,Diesel,8852 Altendorf


### Count numbre of rows and columns in the dateframe

In [51]:
# Dimension (rows, columns)
print('Dimension:', df.shape)

# Number of rows
print('Number of rows:', df.shape[0])

# Number of columns
print('Number of columns:', df.shape[1])

Dimension: (1587, 6)
Number of rows: 1587
Number of columns: 6


### Get Data types from Web scraping

In [52]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

web-scraper-order    object
description_raw      object
price_raw            object
mileage_raw          object
fuel_type_raw        object
location_raw         object
dtype: object

#### Extract and save relevant information from raw data using regular expressions (regex)

### Extract Price

In [53]:
# Extract values from 'price_raw' column
price = []
for i in df['price_raw']:
    d1 = re.findall(r'[0-9]+', str(i))
    try:
        d2 = ''.join(d1).replace("'", "").replace("CHF", "").replace(".-", "").strip()
    except:
        d2 = None
    price.append(d2)

# Save as new variable in the pandas data frame
df['price'] = pd.Series(price, dtype="int")
    
# Print first 5 values
print(df[['price_raw', 'price']].head(5))

      price_raw  price
0  CHF 15'900.–  15900
1  CHF 71'900.–  71900
2  CHF 37'800.–  37800
3  CHF 19'900.–  19900
4  CHF 84'500.–  84500


In [54]:
df.dtypes

web-scraper-order    object
description_raw      object
price_raw            object
mileage_raw          object
fuel_type_raw        object
location_raw         object
price                 int64
dtype: object

In [55]:
df.head()

,web-scraper-order,description_raw,price_raw,mileage_raw,fuel_type_raw,location_raw,price
0,1733470796-1,MERCEDES-BENZ Camper,CHF 15'900.–,265'000 km,Diesel,"Mittelstrasse 38, 3613 Steffisburg",15900
1,1733470799-2,CARADO I 449 Édition 15 Fiat Capron 169L/1449,CHF 71'900.–,14'000 km,Diesel,"Zone industrielle Les Portettes 2c, 1312 Eclépens",71900
2,1733470801-3,DETHLEFFS Fiat Ducato,CHF 37'800.–,90'000 km,Diesel,"Rte du St-Bernard 8, 1937 Orsières",37800
3,1733470805-4,FIAT Euro Master Wohnmobil,CHF 19'900.–,187'236 km,Diesel,"Badenerstrasse 84, 8952 Schlieren",19900
4,1733470807-5,VW T6.1 California 2.0 TDI Ocean Edition Liber...,CHF 84'500.–,16'500 km,Diesel,8852 Altendorf,84500


### Extract Mileage

In [56]:
# Extract values from 'mileage_raw' column
mileage = []
for i in df['mileage_raw']:
    d1 = re.findall(r'[0-9]+', str(i))
    try:
        d2 = ''.join(d1).replace("'", "").replace("km", "").replace(".-", "").strip()
    except:
        d2 = None
    mileage.append(d2)

# Save as new variable in the pandas data frame
df['mileage'] = pd.Series(mileage, dtype="int")
    
# Print first 5 values
print(df['mileage_raw'].head(5), '\n')
print(df['mileage'].head(5))

0    265'000 km
1     14'000 km
2     90'000 km
3    187'236 km
4     16'500 km
Name: mileage_raw, dtype: object 

0    265000
1     14000
2     90000
3    187236
4     16500
Name: mileage, dtype: int64


In [57]:
df.dtypes

web-scraper-order    object
description_raw      object
price_raw            object
mileage_raw          object
fuel_type_raw        object
location_raw         object
price                 int64
mileage               int64
dtype: object

### Fuel Type to Dummy

In [60]:
#count frequency of values in column 'fuel_type_raw'
df['fuel_type_raw'].value_counts()

fuel_type_raw
Diesel     1453
Benzin       72
-            61
Elektro       1
Name: count, dtype: int64

In [72]:
# Filter for Diesel and Benzin
df = df[(df['fuel_type_raw'] == 'Diesel') | (df['fuel_type_raw'] == 'Benzin')]

count = df['fuel_type_raw'].value_counts()
print(count)

# One-Hot-Encoding für die Spalte 'fuel_type_raw' and change true and false to 0 and 1
df_encoded = pd.get_dummies(df, columns=['fuel_type_raw'], drop_first=True)

# Konvertiere boolesche Werte in 0 und 1
bool_columns = df_encoded.select_dtypes(include=['bool']).columns
df_encoded[bool_columns] = df_encoded[bool_columns].astype(int)

# Überprüfe das Ergebnis
df_encoded



fuel_type_raw
Diesel    1453
Benzin      72
Name: count, dtype: int64


,web-scraper-order,description_raw,price_raw,mileage_raw,location_raw,price,mileage,fuel_type_raw_Diesel
0,1733470796-1,MERCEDES-BENZ Camper,CHF 15'900.–,265'000 km,"Mittelstrasse 38, 3613 Steffisburg",15900,265000,1
1,1733470799-2,CARADO I 449 Édition 15 Fiat Capron 169L/1449,CHF 71'900.–,14'000 km,"Zone industrielle Les Portettes 2c, 1312 Eclépens",71900,14000,1
2,1733470801-3,DETHLEFFS Fiat Ducato,CHF 37'800.–,90'000 km,"Rte du St-Bernard 8, 1937 Orsières",37800,90000,1
3,1733470805-4,FIAT Euro Master Wohnmobil,CHF 19'900.–,187'236 km,"Badenerstrasse 84, 8952 Schlieren",19900,187236,1
4,1733470807-5,VW T6.1 California 2.0 TDI Ocean Edition Liber...,CHF 84'500.–,16'500 km,8852 Altendorf,84500,16500,1
...,...,...,...,...,...,...,...,...
1582,1733477100-1583,"SUNLIGHT Cliff 600 Adventure Edition, Citroen ...",CHF 52'900.–,11'000 km,8406 Winterthur,52900,11000,1
1583,1733477103-1584,"LAIKA Kosmo V9 Compact, Fiat Ducato 2.3MJ 140PS",CHF 56'900.–,14'500 km,8406 Winterthur,56900,14500,1
1584,1733477106-1585,PEUGEOT 335-120 FL2,CHF 32'777.–,132'000 km,"Sulzer-Allee 41, 8404 Winterthur",32777,132000,1
1585,1733477109-1586,CITROEN 2.2 HDI 12Q,CHF 31'777.–,116'700 km,"Sulzer-Allee 41, 8404 Winterthur",31777,116700,1
